In [10]:
from limix.stats import linear_kinship, multivariate_normal
from numpy import zeros, eye
from numpy import ones, stack
from numpy.random import RandomState
from pandas import DataFrame
from limix.qtl import scan

In [3]:
random = RandomState(2)
# samples size
n = 100
# covariates
offset = ones(n) * random.randn()
age = random.randint(16, 75, n)
M = stack((offset, age), axis=1)
M = DataFrame(stack([offset, age], axis=1), columns=["offset", "age"])
M["sample"] = [f"sample{i}" for i in range(n)]
M = M.set_index("sample")
print(M.head())

           offset   age
sample                 
sample0 -0.416758  38.0
sample1 -0.416758  59.0
sample2 -0.416758  34.0
sample3 -0.416758  27.0
sample4 -0.416758  56.0


In [6]:
G = random.randn(n, 4)

In [7]:
# sampling the phenotype
alpha = random.randn(2)
beta = random.randn(4)
eps = random.randn(n)
y = M @ alpha + G @ beta + eps

In [13]:
# Whole genotype of each sample.
X = random.randn(n, 50)
print(type(X))
# Estimate a kinship relationship between samples.
K = linear_kinship(X, verbose=False) + 1e-9 * eye(n)
# Update the phenotype
y += multivariate_normal(random, zeros(n), K)

<class 'numpy.ndarray'>


In [12]:
r = scan(X, y, "normal", K, 𝙼=M, verbose=False)
print(r)

Hypothesis 0
------------

𝐲 ~ 𝓝(𝙼𝜶, 1.226⋅𝙺 + 6.642⋅𝙸)

M     = ['offset' 'age']
𝜶     = [-0.45776409 -0.66305383]
se(𝜶) = [1.84714408 0.01580719]
lml   = -244.00359493370092

Hypothesis 2
------------

𝐲 ~ 𝓝(𝙼𝜶 + G𝛃, s(1.226⋅𝙺 + 6.642⋅𝙸))

          lml       cov. effsizes   cand. effsizes
--------------------------------------------------
mean   -2.435e+02      -5.761e-01       -3.961e-02
std     6.196e-01       1.588e-01        3.206e-01
min    -2.440e+02      -1.167e+00       -6.897e-01
25%    -2.440e+02      -6.635e-01       -2.405e-01
50%    -2.437e+02      -6.622e-01       -8.396e-02
75%    -2.434e+02      -4.610e-01        2.160e-01
max    -2.414e+02      -1.778e-01        7.842e-01

Likelihood-ratio test p-values
------------------------------

       𝓗₀ vs 𝓗₂ 
----------------
mean   4.753e-01
std    2.885e-01
min    2.281e-02
25%    2.601e-01
50%    4.644e-01
75%    7.529e-01
max    9.904e-01
